In [2]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter

In [5]:
PROTOTXT = "weights/MobileNetSSD_deploy.prototxt"
MODEL = "weights/MobileNetSSD_deploy.caffemodel"
GPU_SUPPORT = 0

CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat", "bottle", "bus",  "car", "cat", "chair", "cow", 
           "diningtable", "dog", "horse", "motorbike", "person", "pottedplant", "sheep", "sofa", "train", "tvmonitor"]
COLORS = np.random.uniform(0, 255, size=(len(CLASSES), 3))

net = cv2.dnn.readNetFromCaffe(PROTOTXT, MODEL)
if GPU_SUPPORT:
    net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
    net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
    
image = cv2.imread("images_videos/image3.jpg")
(h, w) = image.shape[:2]
#blob = cv2.dnn.blobFromImage(image, 0.007843, (300, 300), 127.5)
blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)),0.007843, (300,300), 127.5)

net.setInput(blob)
detections = net.forward()
detect_list=list()

for i in np.arange(0, detections.shape[2]):
    confidence = detections[0, 0, i, 2]
    if confidence > 0.6:
        idx = int(detections[0, 0, i, 1])
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")
        label = "{}: {:.2f}%".format(CLASSES[idx],confidence*100)
        label_2 = "{}".format(CLASSES[idx])
        detect_list.append(label_2)
        cv2.rectangle(image, (startX, startY), (endX, endY),COLORS[idx], 2)
        y = startY - 15 if startY - 15 > 15 else startY + 15
        cv2.putText(image, label, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS[idx], 2)
        #print(label_2)
cv2.imshow("object detection", image)
cv2.imwrite("image_object_detected.jpg", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

count = Counter(detect_list)
my_count = pd.Series(detect_list).value_counts()
print("Detected items: \n",my_count.to_string())

Detected items: 
 person    4
